In [1]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent
os.environ["OPENAI_API_KEY"] = "ENter your API key here"

warnings.filterwarnings("ignore")

In [3]:

llm=OpenAI(temperature=0,
           model_name="gpt-3.5-turbo-16k-0613")

### Defining the custom Tools

In [4]:
import yfinance as yf

# Fetch stock data from Yahoo Finance

def get_stock_price(ticker,history=5):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    # print(df.columns)
    
    return df.to_string()

print(get_stock_price("TITAN"))

                  Close   Volume
Date                            
2023-07-31  3004.050049   656048
2023-08-01  3001.899902   602288
2023-08-02  2977.149902  1152335
2023-08-03  2902.100098  3319847
2023-08-04  2904.399902   709334


In [5]:
# Script to scrap top5 googgle news for given company name

def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}&cr=countryIN"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):
    # time.sleep(4) #To avoid rate limit error
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)


    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news


print(get_recent_stock_news("Asian paints"))

Recent News:

0. Asian Paint share price Today Live Updates : Asian Paint closed today at 
₹3339.9, up 0.13% from yesterday's ₹3335.55 | Mint
1. Asian Paint share price Today Live Updates : Asian Paint closed today at 
₹3335.55, down -0.82% from yesterday's ₹3363.25 | Mint



In [8]:
# Fetch financial statements from Yahoo Finance

def get_financial_statements(ticker):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    ticker=ticker+".NS"    
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]    # Remove 4th years data
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    
    # cash_flow = company.cash_flow.to_string()
    # print(balance_sheet)
    # print(cash_flow)
    return balance_sheet
print(get_financial_statements("TATAPOWER.NS"))

                                                          2023-03-31       2022-03-31      2021-03-31
Ordinary Shares Number                                  3195339547.0     3196991847.0    3195339547.0
Share Issued                                            3195339547.0     3196991847.0    3195339547.0
Net Debt                                              447846700000.0   445128500000.0  393856600000.0
Total Debt                                            529230000000.0   511952100000.0  467054800000.0
Tangible Book Value                                   255477800000.0   192170700000.0  191818400000.0
Invested Capital                                      777618600000.0   700316500000.0  654904300000.0
Working Capital                                      -130012100000.0  -141013900000.0 -114893300000.0
Net Tangible Assets                                   255477800000.0   192170700000.0  191818400000.0
Capital Lease Obligations                              39485700000.0    3605120000

### Custom tools

In [7]:
from langchain.tools import DuckDuckGoSearchRun
search=DuckDuckGoSearchRun()

search("Stock news India")

'Share Market Today - Stock Market and Share Market Live Updates: Get all the latest share market and India stock market news and updates on Moneycontrol.com CAMS Q1 Results: Profit grows 18% to Rs 76 crore. "The Indian mutual fund industry grew by 6.4 per cent quarter-on-quarter (Q-o-Q) to reach Rs 43.8 lakh crore in AUM and is well-positioned for future growth. Our assets under management grew by 6.8 per cent Q-o-Q, ahead of the industry, to cross the Rs 30 lakh crore mark,\'\' Anuj Kumar, Managing ... Morgan Stanley upgrades India stocks, reveals names to buy \'Stark contrast\' to China: Morgan Stanley upgrades India stocks and shares 4 top picks Published Thu, Aug 3 20237:47 PM EDT Weizhen... NSE - National Stock Exchange of India Ltd: Live Share/Stock Market News & Updates, Quotes- Nseindia.com NSE Indices Equities Derivatives Reference Rates Market Turnover Complaints & Reporting Filter Indices View All Indices NIFTY 50 19,517.00 135.35 (0.70%) NIFTY NEXT 50 44,765.00 350.45 (0.7

In [42]:
# Making tool list

tools=[
    Tool(
        name="get stock data",
        func=get_stock_price,
        description="Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it "
    ),
    Tool(
        name="DuckDuckGo Search",
        func=search.run,
        description="Use only when you need to get NSE/BSE stock ticker from internet, you can also get recent stock related news. Dont use it for any other analysis or task"
    ),
    Tool(
        name="get recent news",
        func=get_recent_stock_news,
        description="Use this to fetch recent news about stocks"
    ),

    Tool(
        name="get financial statements",
        func=get_financial_statements,
        description="Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it"
    ) 


]

#### Approach1- React Agent

In [74]:
from langchain.agents import initialize_agent 

zero_shot_agent=initialize_agent(
    llm=llm,
    agent="zero-shot-react-description",
    tools=tools,
    verbose=True,
    max_iteration=4,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)



In [75]:
print(zero_shot_agent.agent.llm_chain.prompt.template) 


Answer the following questions as best you can. You have access to the following tools:

get stock data: Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it 
DuckDuckGo Search: Use only when you need to get NSE/BSE stock ticker from internet, you can also get recent stock related news. Dont use it for any other analysis or task
get recent news: Use this to fetch recent news about stocks
get financial statements: Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [get stock data, DuckDuckGo Search, get recent news, get financial statements]
Action Input: the input to the action
Observation: the result of the action
... (t

In [82]:
#Adding predefine evaluation steps in the agent Prompt

new_prompt="""You are a financial advisor. Give stock recommendations for given query.
Everytime first you should identify the company name and get the stock ticker symbole for indian stock.
Answer the following questions as best you can. You have access to the following tools:

get stock data: Use when you are asked to evaluate or analyze a stock. This will output historic share price data. You should input the the stock ticker to it 
DuckDuckGo Search: Use only when you need to get NSE/BSE stock ticker from internet, you can also get recent stock related news. Dont use it for any other analysis or task
get recent news: Use this to fetch recent news about stocks
get financial statements: Use this to get financial statement of the company. With the help of this data companys historic performance can be evaluaated. You should input stock ticker to it

steps- 
Note- if you fail in satisfying any of the step below, Just move to next one
1) Get the company name and search for the "company name + NSE/BSE stock ticker" on internet. Dont hallucinate extract stock ticker as it is from the text. Output- stock ticker
2) Use "get stock data" tool to gather stock info. Output- Stock data
3) Get company's historic financial data using "get financial statements". Output- Financial statement
4) Use this "get recent news" tool to search for latest stock realted news. Output- Stock news
5) Analyze the stock based on gathered data and give detail analysis for investment choice. provide numbers and reasons to justify your answer. Output- Detailed stock Analysis

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do, Also try to follow steps mentioned above
Action: the action to take, should be one of [get stock data, DuckDuckGo Search, get recent news, get financial statements]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question
Begin!

Question: {input}
Thought:{agent_scratchpad}"""



In [83]:
#Setting up new prompt
zero_shot_agent.agent.llm_chain.prompt.template=new_prompt


In [78]:
zero_shot_agent("Is Bajaj Finance a good investment choice right now?")



> Entering new  chain...


I need to find the stock ticker symbol for Bajaj Finance and gather stock data, financial statements, and recent news to analyze its investment potential.
Action: DuckDuckGo Search
Action Input: Bajaj Finance NSE/BSE stock ticker
Observation: Bajaj Finance Limited Share Price Today, Live NSE Stock Price: Get the latest Bajaj Finance Limited news, company updates, quotes, offers, annual financial reports, graph, volumes, 52 week high low, buy sell tips, balance sheet, historical charts, market performance, capitalisation, dividends, volume, profit and loss account, research, results and more details at NSE India. Housing Development Finance Corporation ₹2,724.30 0.00 (0.00%) Bajaj Holdings & Investment ₹7,400.90 -142.60 (-1.89%) Bajaj Finserv ₹1,587.60 -10.70 (0.67%) HDFC Asset Management Company... In this regard, the Company has sent a detailed communication to all the Shareholders holding shares as on Friday, 9 June 2023 whose email addresses are registered with the Company/Depositor

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


I have gathered the necessary data for Bajaj Finance. Now I can analyze its investment potential.
Action: get recent news
Action Input: Bajaj Finance
Observation: Recent News:

0. Bajaj Finance share price Today Live Updates : Bajaj Finance closed today 
at ₹7145.05, up 0.52% from yesterday's ₹7107.95 | Mint
1. Recos Updates: Motilal Oswal Financial Services Recommends Bajaj Finance 
with 23.46% Upside Potential

Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/ac

Based on the gathered data, I can now provide a detailed analysis of Bajaj Finance as an investment choice.

Bajaj Finance (NSE: BAJFINANCE) is a finance company listed on the National Stock Exchange of India. The stock has shown a recent upward trend, with a closing price of ₹7,145.05, up 0.52% from the previous day's closing price of ₹7,107.95. The stock has a 52-week low of ₹5,485.70 and a 52-week high of ₹7,999.90.

In terms of financial performance, Bajaj Finance has shown consistent growth in its net tangible assets, common stock equity, and total equity over the past four years. The company's net debt has also increased over the same period, indicating a higher level of leverage. However, Bajaj Finance has a strong capitalization and invested capital, which suggests a solid financial position.

Recent news about Bajaj Finance includes a recommendation from Motilal Oswal Financial Services, which suggests a 23.46% upside potential for the stock.

Based on the analysis of the stoc

{'input': 'Is Bajaj Finance a good investment choice right now?',
 'output': 'Bajaj Finance is a good investment choice right now based on the analysis of stock data, financial statements, and recent news.',
 'intermediate_steps': [(AgentAction(tool='DuckDuckGo Search', tool_input='Bajaj Finance NSE/BSE stock ticker', log='I need to find the stock ticker symbol for Bajaj Finance and gather stock data, financial statements, and recent news to analyze its investment potential.\nAction: DuckDuckGo Search\nAction Input: Bajaj Finance NSE/BSE stock ticker'),
   'Bajaj Finance Limited Share Price Today, Live NSE Stock Price: Get the latest Bajaj Finance Limited news, company updates, quotes, offers, annual financial reports, graph, volumes, 52 week high low, buy sell tips, balance sheet, historical charts, market performance, capitalisation, dividends, volume, profit and loss account, research, results and more details at NSE India. Housing Development Finance Corporation ₹2,724.30 0.00 (0.0

In [84]:
response = zero_shot_agent({"input":"Shall I invest in Asian paints?"})



> Entering new  chain...
I need to find the stock ticker symbol for Asian Paints and gather stock data, financial statements, and recent news to analyze the stock.
Action: DuckDuckGo Search
Action Input: "Asian Paints NSE/BSE stock ticker"
Observation: Asian Paints Limited Share Price Today, Live NSE Stock Price: Get the latest Asian Paints Limited news, company updates, quotes, offers, annual financial reports, graph, volumes, 52 week high low, buy sell tips, balance sheet, historical charts, market performance, capitalisation, dividends, volume, profit and loss account, research, results and more details at NSE India. Asian Paints Share Price Today (02 Aug, 2023) - Asian Paints Share Price Live NSE/BSE Business News › Markets › Stocks › Stock Price Quotes › Asian Paints Share Price Sector: Building Materi... Industry: Paints/Varnishes Asian Paints Share Price 3,348.10 5.15 (0.15 %) Volume: 5,89,686 Live Last Updated On: 02 Aug, 2023, 02:22 PM IST Asian Paints Ltd Share Price Today,

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..


I have gathered the necessary data for Asian Paints. Now I can analyze the stock and provide a detailed analysis for investment choice.
Action: Analyze the stock based on gathered data
Observation: Invalid Format: Missing 'Action Input:' after 'Action:'
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/ac

I need to provide the action input for analyzing the stock based on the gathered data.
Action: Analyze the stock based on gathered data
Action Input: ASIANPAINT
Observation: Analyze the stock based on gathered data is not a valid tool, try another one.
Thought:

...

I need to use a different tool to analyze the stock based on the gathered data.
Action: get financial statements
Action Input: ASIANPAINT
Observation:                                                                           2023-03-31      2022-03-31      2021-03-31      2020-03-31
Treasury Shares Number                                                      357659.0        223240.0             NaN             NaN
Ordinary Shares Number                                                   958840131.0     958974550.0     959197790.0     959197790.0
Share Issued                                                             958840131.0     959197790.0     959197790.0     959197790.0
Net Debt                                                                4491100000.0    1539000000.0      21500000.0             NaN
Total Debt                                                             19326200000.0   15868400000.0   10930800000.0   11182400000.0
Tangible Book Value                                

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/ac

I have gathered the necessary data for Asian Paints. Now I can analyze the stock and provide a detailed analysis for investment choice.
Action: Analyze the stock based on gathered data
Action Input: Stock data, financial statements, recent news
Observation: Analyze the stock based on gathered data is not a valid tool, try another one.
Thought:

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo-16k in organization org-ee1LP900po9H9JaxPmhhwDap on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/ac

KeyboardInterrupt: 

In [ ]:
# The why this approach did not work is added in the github readme
# It is recommended to use the approach2 for robust results

##### Approach 2

In [9]:
import openai

import os
os.environ["OPENAI_API_KEY"] = "Enter your API key here"
openai.api_key="Enter your API key here"


In [10]:
from langchain.llms import OpenAI

llm=OpenAI(temperature=0,
           model_name="gpt-3.5-turbo-16k-0613")

In [11]:
#Openai function calling

import json
function=[
        {
        "name": "get_company_Stock_ticker",
        "description": "This will get the indian NSE/BSE stock ticker of the company",
        "parameters": {
            "type": "object",
            "properties": {
                "ticker_symbol": {
                    "type": "string",
                    "description": "This is the stock symbol of the company.",
                },

                "company_name": {
                    "type": "string",
                    "description": "This is the name of the company given in query",
                }
            },
            "required": ["company_name","ticker_symbol"],
        },
    }
]



def get_stock_ticker(query):
    response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0,
            messages=[{
                "role":"user",
                "content":f"Given the user request, what is the comapany name and the company stock ticker ?: {query}?"
            }],
            functions=function,
            function_call={"name": "get_company_Stock_ticker"},
    )
    message = response["choices"][0]["message"]
    arguments = json.loads(message["function_call"]["arguments"])
    company_name = arguments["company_name"]
    company_ticker = arguments["ticker_symbol"]
    return company_name,company_ticker

get_stock_ticker("How is ths stock of Paytm doing?")

('Paytm', 'PAYTM')

In [12]:
def Anazlyze_stock(query):
    #agent.run(query) Outputs Company name, Ticker
    Company_name,ticker=get_stock_ticker(query)
    print({"Query":query,"Company_name":Company_name,"Ticker":ticker})
    stock_data=get_stock_price(ticker,history=10)
    stock_financials=get_financial_statements(ticker)
    stock_news=get_recent_stock_news(Company_name)

    # available_information=f"Stock Price: {stock_data}\n\nStock Financials: {stock_financials}\n\nStock News: {stock_news}"
    available_information=f"Stock Financials: {stock_financials}\n\nStock News: {stock_news}"

    print("\n\nAnalyzing.....\n")
    analysis=llm(f"Give detail stock analysis, Use the available data and provide investment recommendation. \
             The user is fully aware about the investment risk, dont include any kind of warning like 'It is recommended to conduct further research and analysis or consult with a financial advisor before making an investment decision' in the answer \
             User question: {query} \
             You have the following information available about {Company_name}. Write (5-8) pointwise investment analysis to answer user query, At the end conclude with proper explaination.Try to Give positives and negatives  : \
              {available_information} "
             )
    print(analysis)

    return analysis

In [57]:

out=Anazlyze_stock("Shall I invest in Adani power right now?")    


{'Query': 'Shall I invest in Adani power right now?', 'Company_name': 'Adani Power', 'Ticker': 'ADANIPOWER'}


Analyzing.....

Investment Thesis for Adani Power:

1. Strong Financials: Adani Power has shown consistent growth in its financials over the past three years. The company has a positive tangible book value, indicating a strong asset base. Additionally, the company has a high invested capital, which suggests a strong financial position.

2. Increasing Stock Price: The stock price of Adani Power has been increasing steadily over the past few days. This indicates positive investor sentiment and potential for further growth in the future.

3. Declining Revenues: Adani Power has reported declining revenues in its recent quarterly results. This could be a concern for investors as it may indicate a slowdown in the company's business operations. However, further analysis is required to determine the reasons behind the decline and its potential impact on future performance.

4. Positiv

In [65]:
Anazlyze_stock("Is it a good time to invest in Yes Bank?")    

{'Query': 'Is it a good time to invest in Yes Bank?', 'Company_name': 'Yes Bank', 'Ticker': 'YESBANK'}


Analyzing.....

Investment Thesis for Yes Bank:

1. Financial Performance: Yes Bank has shown improvement in its financials over the past three years. The net debt has increased, indicating higher borrowing, but the tangible book value and common stock equity have also increased, suggesting a stronger financial position.

2. Total Capitalization: The total capitalization of Yes Bank has been consistently increasing, indicating a growing investor base and potential for future growth. This can be seen as a positive sign for investors considering investing in the bank.

3. Total Assets: Yes Bank's total assets have also been increasing steadily, indicating the bank's ability to attract and manage a larger pool of assets. This growth in assets can contribute to the bank's profitability and potential for future expansion.

4. Stock News: Recent news about Yes Bank suggests that the stock

"Investment Thesis for Yes Bank:\n\n1. Financial Performance: Yes Bank has shown improvement in its financials over the past three years. The net debt has increased, indicating higher borrowing, but the tangible book value and common stock equity have also increased, suggesting a stronger financial position.\n\n2. Total Capitalization: The total capitalization of Yes Bank has been consistently increasing, indicating a growing investor base and potential for future growth. This can be seen as a positive sign for investors considering investing in the bank.\n\n3. Total Assets: Yes Bank's total assets have also been increasing steadily, indicating the bank's ability to attract and manage a larger pool of assets. This growth in assets can contribute to the bank's profitability and potential for future expansion.\n\n4. Stock News: Recent news about Yes Bank suggests that the stock has seen a marginal increase in price and has been holding steady. This stability in the stock price can be see

In [13]:
Anazlyze_stock("How vednata share is doing?")    

{'Query': 'How vednata share is doing?', 'Company_name': 'Vedanta', 'Ticker': 'VEDL'}


Analyzing.....

1. The number of ordinary shares for Vedanta has remained relatively stable over the past three years, indicating no significant dilution or increase in share capital.
2. Net debt has increased significantly from 250.48 billion in 2021 to 592.56 billion in 2023, indicating a higher level of debt for the company. This could potentially impact the company's financial stability and ability to generate profits.
3. Tangible book value has decreased from 588.03 billion in 2021 to 351.91 billion in 2023. This indicates a decrease in the company's tangible assets and could be a cause for concern.
4. Working capital has shown a significant improvement from -29.32 billion in 2021 to 28.62 billion in 2022. This indicates that the company has improved its ability to meet short-term obligations and manage its current assets and liabilities.
5. Total equity has remained relatively stable over the 

"1. The number of ordinary shares for Vedanta has remained relatively stable over the past three years, indicating no significant dilution or increase in share capital.\n2. Net debt has increased significantly from 250.48 billion in 2021 to 592.56 billion in 2023, indicating a higher level of debt for the company. This could potentially impact the company's financial stability and ability to generate profits.\n3. Tangible book value has decreased from 588.03 billion in 2021 to 351.91 billion in 2023. This indicates a decrease in the company's tangible assets and could be a cause for concern.\n4. Working capital has shown a significant improvement from -29.32 billion in 2021 to 28.62 billion in 2022. This indicates that the company has improved its ability to meet short-term obligations and manage its current assets and liabilities.\n5. Total equity has remained relatively stable over the past three years, indicating a consistent level of shareholder's equity in the company.\n6. Total l